In [ ]:
import pandas as pd

import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
# Upload the csv into the pandas dataframe and see the top 5 data
from google.colab import files
uploaded = files.upload()

Saving XRP Jun - Sep 2021.csv to XRP Jun - Sep 2021.csv


In [ ]:
# Input the csv file into a dataframe, with date being its index column
df = pd.read_csv("Token.csv", index_col="created", parse_dates=True)
df.head()

,Unnamed: 0,text
created,,
2021-07-01,1,RT @MemesZil: TODAY’S #GIVEAWAY ????\n\n?? Pri...
2021-07-01,2,RT @coinpricenow: 1 Bitcoin ( #BTC )\nDollar: ...
2021-07-01,3,RT @MemesZil: TODAY’S #GIVEAWAY ????\n\n?? Pri...
2021-07-01,4,"1 Bitcoin ( #BTC )\nDollar: 33,516.10$ \n\n1 B..."
2021-07-01,5,RT @MemesZil: TODAY’S #GIVEAWAY ????\n\n?? Pri...


In [ ]:
# Clean the text

# Import libraries
import re

# Create a function to clean the tweets
def cleanRT(text):
    text = re.sub(r'RT[\s]+', '', text) # Remove RT    
    return text

def cleanURLExcessMentions(text):
    text = re.sub(r'https?://[A-Za-z0-9.]+', '', text) # Remove hyperlink
    text = re.sub(r'@[A-Za-z0-9_]+', '', text) # Remove mentions
    text = re.sub(r'#[A-Za-z0-9_]+', '', text) # Remove the '#' symbol
    text = re.sub(r'$[A-Za-z0-9_]+', '', text) # Remove the '$' symbol
    return text

def cleanOtherSymbol(text):
    text = re.sub(r'[$&()!?:.]', '', text) # Remove the symbols
    text = re.sub(r'\n', '', text) # Remove additional symbols 
    return text

In [ ]:
# Cleaning the text
df["text"] = df['text'].apply(cleanRT)

df.head()

,Unnamed: 0,text
created,,
2021-07-01,1,@MemesZil: TODAY’S #GIVEAWAY ????\n\n?? Prize:...
2021-07-01,2,"@coinpricenow: 1 Bitcoin ( #BTC )\nDollar: 33,..."
2021-07-01,3,@MemesZil: TODAY’S #GIVEAWAY ????\n\n?? Prize:...
2021-07-01,4,"1 Bitcoin ( #BTC )\nDollar: 33,516.10$ \n\n1 B..."
2021-07-01,5,@MemesZil: TODAY’S #GIVEAWAY ????\n\n?? Prize:...


In [ ]:
# Cleaning the text
df["text"] = df['text'].apply(cleanURLExcessMentions)

df.head()

,Unnamed: 0,text
created,,
2021-07-01,1,: TODAY’S ????\n\n?? Prize: 100 $zil\n⏰ Ends:...
2021-07-01,2,": 1 Bitcoin ( )\nDollar: 33,516.10$ \n\n1 Bit..."
2021-07-01,3,: TODAY’S ????\n\n?? Prize: 100 $zil\n⏰ Ends:...
2021-07-01,4,"1 Bitcoin ( )\nDollar: 33,516.10$ \n\n1 Bitco..."
2021-07-01,5,: TODAY’S ????\n\n?? Prize: 100 $zil\n⏰ Ends:...


In [ ]:
# Cleaning the text
df["text"] = df['text'].apply(cleanOtherSymbol)

df.head()

,Unnamed: 0,text
created,,
2021-07-01,1,TODAY’S Prize 100 zil⏰ Ends ~24 hrsEASY to ...
2021-07-01,2,"1 Bitcoin Dollar 33,51610 1 Bitcoin Cash ..."
2021-07-01,3,TODAY’S Prize 100 zil⏰ Ends ~24 hrsEASY to ...
2021-07-01,4,"1 Bitcoin Dollar 33,51610 1 Bitcoin Cash D..."
2021-07-01,5,TODAY’S Prize 100 zil⏰ Ends ~24 hrsEASY to ...


In [ ]:
df.shape

(119130, 2)

In [ ]:
# Remove keywords with Giving, giving or giveaway
searchfor = ['Giving', 'giving', 'Giveaway', 'giveaway', 'Airdrop', 'airdrop', 'GIVEAWAY']

df = df[~df.text.str.contains('|'.join(searchfor))] # Remove rows with string from searchfor
# df = df.drop_duplicates(subset = None, keep = 'first', inplace = False) # Remove duplicate

df.head()

,Unnamed: 0,text
created,,
2021-07-01,1,TODAY’S Prize 100 zil⏰ Ends ~24 hrsEASY to ...
2021-07-01,2,"1 Bitcoin Dollar 33,51610 1 Bitcoin Cash ..."
2021-07-01,3,TODAY’S Prize 100 zil⏰ Ends ~24 hrsEASY to ...
2021-07-01,4,"1 Bitcoin Dollar 33,51610 1 Bitcoin Cash D..."
2021-07-01,5,TODAY’S Prize 100 zil⏰ Ends ~24 hrsEASY to ...


In [ ]:
df.shape

(115752, 2)

In [ ]:
# Create a function to compute the negative, neutral, and positive analysis using vader
df['scores'] = df['text'].apply(lambda text: sid.polarity_scores(text))
df.head()

,Unnamed: 0,text,scores
created,,,
2021-07-01,1,TODAY’S Prize 100 zil⏰ Ends ~24 hrsEASY to ...,"{'neg': 0.0, 'neu': 0.798, 'pos': 0.202, 'comp..."
2021-07-01,2,"1 Bitcoin Dollar 33,51610 1 Bitcoin Cash ...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
2021-07-01,3,TODAY’S Prize 100 zil⏰ Ends ~24 hrsEASY to ...,"{'neg': 0.0, 'neu': 0.798, 'pos': 0.202, 'comp..."
2021-07-01,4,"1 Bitcoin Dollar 33,51610 1 Bitcoin Cash D...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
2021-07-01,5,TODAY’S Prize 100 zil⏰ Ends ~24 hrsEASY to ...,"{'neg': 0.0, 'neu': 0.798, 'pos': 0.202, 'comp..."


In [ ]:
# Create compounded score of the initial neg, neu, and pos data
df['compound'] = df['scores'].apply(lambda score_dict: score_dict['compound'])
df['sentiment_type'] = ''
df.loc[df.compound > 0, 'sentiment_type'] = 'POSITIVE'
df.loc[df.compound == 0, 'sentiment_type'] = 'NEUTRAL'
df.loc[df.compound < 0, 'sentiment_type'] = 'NEGATIVE'
df.head()

,Unnamed: 0,text,scores,compound,sentiment_type
created,,,,,
2021-07-01,1,TODAY’S Prize 100 zil⏰ Ends ~24 hrsEASY to ...,"{'neg': 0.0, 'neu': 0.798, 'pos': 0.202, 'comp...",0.5106,POSITIVE
2021-07-01,2,"1 Bitcoin Dollar 33,51610 1 Bitcoin Cash ...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,NEUTRAL
2021-07-01,3,TODAY’S Prize 100 zil⏰ Ends ~24 hrsEASY to ...,"{'neg': 0.0, 'neu': 0.798, 'pos': 0.202, 'comp...",0.5106,POSITIVE
2021-07-01,4,"1 Bitcoin Dollar 33,51610 1 Bitcoin Cash D...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,NEUTRAL
2021-07-01,5,TODAY’S Prize 100 zil⏰ Ends ~24 hrsEASY to ...,"{'neg': 0.0, 'neu': 0.798, 'pos': 0.202, 'comp...",0.5106,POSITIVE


In [ ]:
# Download the csv file as excel to local machine
df.to_csv("Token.csv")
files.download("Token.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>